In [1]:
# download from https://commonvoice.mozilla.org/en/datasets
# !tar -zxf cv-corpus-10.0-2022-07-04-id.tar.gz
# !rm cv-corpus-10.0-2022-07-04-id.tar.gz

In [2]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part2.tar
# !mkdir part1-v2
# !tar -xf semisupervised-26-02-2021-part2.tar -C part1-v2
# !rm semisupervised-26-02-2021-part2.tar

In [3]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part3.tar
# !mkdir part2-v2
# !tar -xf semisupervised-26-02-2021-part3.tar -C part2-v2
# !rm semisupervised-26-02-2021-part3.tar

In [4]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-26-02-2021-part4.tar
# !mkdir part3-v2
# !tar -xf semisupervised-26-02-2021-part4.tar -C part3-v2
# !rm semisupervised-26-02-2021-part4.tar

In [5]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part1.tar
# !tar -xf semisupervised-24-03-2021-part1.tar
# !rm semisupervised-24-03-2021-part1.tar

In [6]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part2.tar
# !tar -xf semisupervised-24-03-2021-part2.tar
# !rm semisupervised-24-03-2021-part2.tar

In [7]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech/semisupervised-24-03-2021-part3.tar
# !tar -xf semisupervised-24-03-2021-part3.tar
# !rm semisupervised-24-03-2021-part3.tar

In [8]:
# !tar -zxf sani-clean.tar.gz
# !tar -xf sani-clean.tar.gz
# !rm sani-clean.tar.gz

In [9]:
# https://huggingface.co/datasets/google/fleurs/blob/main/fleurs.py

# !wget https://storage.googleapis.com/xtreme_translations/FLEURS102/ms_my.tar.gz
# !tar -zxf ms_my.tar.gz
# !rm ms_my.tar.gz

In [10]:
# https://huggingface.co/datasets/google/fleurs/blob/main/fleurs.py

# !wget https://storage.googleapis.com/xtreme_translations/FLEURS102/id_id.tar.gz
# !tar -zxf id_id.tar.gz
# !rm id_id.tar.gz

In [11]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/speech-bahasa.zip
# !unzip speech-bahasa.zip
# !rm speech-bahasa.zip

In [12]:
# !wget https://f000.backblazeb2.com/file/malay-dataset/streaming.zip -O wikipedia-asr.zip
# !unzip wikipedia-asr.zip
# !rm wikipedia-asr.zip

In [13]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/news-speech.zip
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/collections/transcript-news.json
# !unzip news-speech.zip -d news
# !rm news-speech.zip

In [14]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/trainset-audiobook.tar.gz
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/text-audiobook.tar.gz
# !tar -xf trainset-audiobook.tar.gz
# !tar -zxf text-audiobook.tar.gz
# !rm trainset-audiobook.tar.gz text-audiobook.tar.gz

In [15]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/asr-dataset/malay-test.json
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/asr-dataset/malay-test.tar.gz
# !tar -zxf malay-test.tar.gz
# !rm malay-test.tar.gz

In [16]:
# !wget https://f000.backblazeb2.com/file/malaya-speech-model/data/Haqkiem_8hour_audio_text.zip
# !mkdir haqkiem
# !unzip Haqkiem_8hour_audio_text.zip -d haqkiem

In [17]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [18]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import json

In [19]:
base_directory = os.getcwd()

In [20]:
!ls

audio-iium.zip
audio-wattpad.zip
benchmark-google-speech-malay-dataset.ipynb
clean
combined
convert-16k.ipynb
cv-corpus-10.0-2022-07-04
evaluate-best-rq-conformer-large-fleurs102.ipynb
evaluate-best-rq-conformer-tiny-fleurs102.ipynb
evaluate-conformer-fleurs102.ipynb
evaluate-hubert-conformer-fleurs102.ipynb
evaluate-hubert-conformer-large-fleurs102.ipynb
evaluate-hubert-conformer-tiny-fleurs102.ipynb
evaluate-tiny-conformer-fleurs102.ipynb
haqkiem
Haqkiem_8hour_audio_text.zip
id
id_id
iium
__MACOSX
malay-asr-test.json
malay-asr-train.json
malay-audio-test.txt
malay-audio.txt
malay-test
malay-test.json
ms
ms_my
news
part1
part1-v2
part2
part2-v2
part3
part3-v2
prepare-stt.ipynb
prepare-stt-test-set.ipynb
prepare-train-valid-set-fairseq-wav2vec2.ipynb
sebut-perkataan-man
sebut-perkataan-woman
semisupervised-voxlingua107.ipynb
shuffled-iium.json
streaming
test.wav
text-audiobook
tolong-sebut
transcript-news.json
transcript-wattpad.json
wattpad
wav2vec2-malay


In [21]:
!ls ms_my/audio

dev  test  train


In [22]:
ms_my_train = pd.read_csv('ms_my/train.tsv', sep = '\t', header=None)
ms_my_dev = pd.read_csv('ms_my/dev.tsv', sep = '\t', header=None)

In [23]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)
tokenizer.tokenize(ms_my_train.iloc[3, 3])

['dalam',
 'tahun',
 '-',
 'tahun',
 'terakhir',
 'abad',
 'ke',
 '-',
 '18',
 'vietnam',
 'juga',
 'telah',
 'menjajah',
 'kemboja']

In [24]:
def tokenize_and_replace(t):
    tokenized = tokenizer.tokenize(t)
    for i in range(len(tokenized)):
        c = cardinal(tokenized[i])
        if c != tokenized[i]:
            tokenized[i] = c
    return ' '.join(tokenized)

tokenize_and_replace(ms_my_train.iloc[3, 3])

'dalam tahun - tahun terakhir abad ke - lapan belas vietnam juga telah menjajah kemboja'

In [25]:
ms_my = []
for i in range(len(ms_my_train)):
    a = ms_my_train.iloc[i, 1]
    t = ms_my_train.iloc[i, 3]
    t = tokenize_and_replace(t)
    
    ms_my.append((f'{base_directory}/ms_my/audio/train/{a}', t))
    
for i in range(len(ms_my_dev)):
    a = ms_my_dev.iloc[i, 1]
    t = ms_my_dev.iloc[i, 3]
    t = tokenize_and_replace(t)
    
    ms_my.append((f'{base_directory}/ms_my/audio/dev/{a}', t))

In [26]:
len(ms_my)

2991

In [27]:
df = pd.read_csv(f'{base_directory}/cv-corpus-10.0-2022-07-04/id/validated.tsv', sep = '\t')
df = df[(df['sentence'].str.len() > 5) & (df['sentence'].str.count(' ') > 0)]
print(df.shape)

(22995, 10)


In [28]:
id_commonvoice = []
for i in range(len(df)):
    p = f"{base_directory}/cv-corpus-10.0-2022-07-04/id/clips/{df['path'].iloc[i]}"
    t = df['sentence'].iloc[i]
    if len(t) < 5:
        continue
    id_commonvoice.append((p, t))

len(id_commonvoice)

22995

In [29]:
malay = glob(f'{base_directory}/part*/output-wav/*.wav')
len(malay)

1008453

In [30]:
malay.extend(glob(f'{base_directory}/part*/semisupervised/output-wav/*.wav'))
len(malay)

1364344

In [31]:
import re

In [32]:
khalil = glob(f'{base_directory}/tolong-sebut/*.wav')
mas = glob(f'{base_directory}/sebut-perkataan-woman/*.wav')
husein = glob(f'{base_directory}/sebut-perkataan-man/*.wav')
len(khalil), len(mas), len(husein)

(565, 200, 698)

In [33]:
khalils = []
for i in tqdm(khalil):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'tolong sebut {t}'
        khalils.append((i, text))
    except Exception as e:
        print(e)

100%|██████████████████████████████████████| 565/565 [00:00<00:00, 20247.45it/s]


In [34]:
mass = []
for i in tqdm(mas):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'sebut perkataan {t}'
        mass.append((i, text))
    except Exception as e:
        print(e)


100%|████████████████████████████████████| 200/200 [00:00<00:00, 1632024.90it/s]


In [35]:
huseins = []
for i in tqdm(husein):
    try:
        t = i.split('/')[-1].replace('.wav','')
        text = f'sebut perkataan {t}'
        huseins.append((i, text))
    except Exception as e:
        print(e)

100%|████████████████████████████████████| 698/698 [00:00<00:00, 1902289.92it/s]


In [ ]:
malays = []
for i in tqdm(malay):
    try:
        p = i.replace('output-wav','output-text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 3:
            continue
        malays.append((i, text))
    except Exception as e:
        print(e)

  2%|▍                               | 21229/1364344 [03:54<62:38:12,  5.96it/s]

In [ ]:
wikipedia = []
wavs = glob(f'{base_directory}/streaming/*wav')
for i in tqdm(wavs):
    text = os.path.split(i)[1].replace('.wav', '')
    wikipedia.append((i, text))
    
len(wikipedia)

In [ ]:
news = []
wavs = glob(f'{base_directory}/news/audio/*wav')

with open(f'{base_directory}/transcript-news.json') as fopen:
    transcript_news = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript_news[int(index)]
    news.append((i, text))

In [ ]:
audiobook = []
wavs = glob(f'{base_directory}/combined/*wav')
for i in tqdm(wavs):
    t = '/'.join(i.split('<>')[1:])
    t = t.split('.wav')[0]
    t = t.replace('output-wav', 'output-text')
    with open(f'{base_directory}/text-audiobook/{t}.wav.txt') as fopen:
        text = fopen.read()
    audiobook.append((i, text))

In [ ]:
df = pd.read_csv(f'{base_directory}/haqkiem/metadata.csv', header = None, sep = '|')
txts = df.values.tolist()
haqkiem = []
for f in tqdm(txts):
    text = f[1]
    text = text.split('.,,')[0]
    f = f[0]
    r = f'{base_directory}/haqkiem/{f}.wav'
    haqkiem.append((r, text))

In [ ]:
sani = glob(f'{base_directory}/clean/**/*.wav', recursive = True)
sanis = []
for i in tqdm(sani):
    with open(i[:-4] + '.txt') as fopen:
        text = fopen.read()
    if len(text):
        sanis.append((i, text))
    
len(sanis)

In [ ]:
wattpad = []
wavs = glob(f'{base_directory}/wattpad/audio-wattpad/*wav')

with open(f'{base_directory}/transcript-wattpad.json') as fopen:
    transcript = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript[int(index)]
    wattpad.append((i, text))
    
len(wattpad)

In [ ]:
iium = []
wavs = glob(f'{base_directory}/iium/audio-iium/*wav')

with open(f'{base_directory}/shuffled-iium.json') as fopen:
    transcript = json.load(fopen)
    
for i in tqdm(wavs):
    index = i.split('/')[-1].replace('.wav','')
    text = transcript[int(index)]
    iium.append((i, text))
    
len(iium)

In [ ]:
audios = id_commonvoice + malays + wikipedia + news + audiobook + haqkiem + khalils + mass + huseins + ms_my + sanis
audios, texts = zip(*audios)

In [ ]:
len(texts)

In [ ]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [ ]:
processed_text = [preprocessing_text(t) for t in tqdm(texts)]

In [ ]:
filtered_audios, filtered_texts = [], []
for i in tqdm(range(len(processed_text))):
    if len(processed_text[i]) and not len(re.findall(r'\d+', processed_text[i])):
        filtered_audios.append(audios[i])
        filtered_texts.append(processed_text[i])

In [ ]:
len(filtered_audios), len(filtered_audios) / len(audios)

In [ ]:
with open('malay-asr-train.json', 'w') as fopen:
    json.dump({'X': filtered_audios, 'Y':filtered_texts}, fopen)

In [ ]:
filtered_audios[0]

In [ ]:
with open('malay-audio.txt', 'w') as fopen:
    fopen.write('\n'.join(filtered_audios))